# Mapping multiple organizations with getorg
This generates the data that is used for [this markercluster leaflet HTML/Javascript page](http://getorg.github.io/jupyter_map/), which runs entirely on the viewer's web browser. 
## Installation
We need a few packages for this, which can be installed with pip:

In [1]:
!pip install getorg --upgrade
!pip install ipyleaflet
!jupyter nbextension enable --py ipyleaflet

Requirement already up-to-date: getorg in /home/vm/anaconda3/lib/python3.5/site-packages
Requirement already up-to-date: geopy in /home/vm/anaconda3/lib/python3.5/site-packages (from getorg)
Requirement already up-to-date: retrying in /home/vm/anaconda3/lib/python3.5/site-packages (from getorg)
Requirement already up-to-date: pygithub in /home/vm/anaconda3/lib/python3.5/site-packages (from getorg)
Requirement already up-to-date: six>=1.7.0 in /home/vm/anaconda3/lib/python3.5/site-packages (from retrying->getorg)
Unrecognized JSON config file version, assuming version 1
Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [2]:
import getorg
from github import Github
import ghlogin
gh = Github(login_or_token=ghlogin.gh_key)

IPywidgets and ipyleaflet support enabled.


### Example: querying Jupyter and IPython organizations
We first create a list containing strings of all the organizations. You can also just send a single organization, in either a list of one or just a string.

In [3]:
orgs = ["jupyter","ipython","jupyter-attic","jupyterhub"]

Then we call the method, with the github object, list of organizations, and optional debug and excluded username parameters. Note: it takes 20-30 minutes of 'wall time' to get all the contributor locations for each of these organizations.

In [4]:
org_map, org_locations, org_metadata = getorg.orgmap.map_orgs(gh, orgs, debug=2)


Querying organization Project Jupyter

Querying repository nbviewer
............................................
Querying repository nbconvert-examples
.....
Querying repository colaboratory
.............
Querying repository jupyter.github.io
...................
Querying repository design
.........
Querying repository nbcache
..
Querying repository nbgrader
............
Querying repository tmpnb
.....................
Querying repository nature-demo
......
Querying repository jupyter-drive
.........
Querying repository tmpnb-redirector
...
Querying repository tmpnb-deploy
...
Querying repository docker-demo-images
......................
Querying repository try.jupyter.org
...
Querying repository strata-sv-2015-tutorial
..
Querying repository testpath
.
Querying repository scipy-2015-advanced-topics
.
Querying repository jupyter_core
..............
Querying repository nbformat
........................
Querying repository jupyter_client
..................................................


In [5]:
org_metadata

{'duplicate_count': 592,
 'error_count': 0,
 'no_loc_count': 650,
 'user_loc_count': 731}

## Displaying the map
In a Jupyter notebook with ipyleaflet support, we can display the map by referring to the org_map object. Note that these do not always render well if you are not running a notebook on a Jupyter server (i.e. if you are viewing this on github or nbviewer), so I've added a screenshot image to show what it looks like.

In [6]:
org_map

![jupyter map](images/jupyter-map.png)

You can export this to embedable HTML files with the Widgets->Embed Widgets code in the Jupyter notebook menu bar.

## Writing the map to a file


### location_dict_to_csv
This outputs username/username hash, longitude, and latitude to a CSV file.

In [8]:
getorg.orgmap.location_dict_to_csv(org_locations, "jupyter-locations.csv", hashed_usernames=True)

'Written to jupyter-locations.csv'

### output_html_cluster_map
This generates a javascript variable file with all the location data, an HTML page to display a leaflet markercluster map, and all of the leaflet dependencies in a folder. You can open map.html to get a dynamic clustered map.

In [9]:
getorg.orgmap.output_html_cluster_map(org_locations,"jupyter_cluster_map")

'Written map to jupyter_cluster_map/'

You can see the results in [this webpage](https://getorg.github.io/jupyter_map/), which I've screenshotted below:
![jupytermap](images/markercluster.png)